# The useMemo Hook

## WebProgrammer 
## Week 4: React

### Antonio Pisanello - Nomades

# Introduction

Le hook `useMemo` permet de mémoriser le résultat d'une fonction pour éviter de la recalculer à chaque rendu. Il est utile pour optimiser les performances de votre application React.

# Initial code

```jsx
import { useState, useEffect, useMemo } from 'react'
import { Input } from './components/forms/Input'
import { Checkbox } from './components/forms/Checkbox'

function App() {
  const [username, setUsername] = useState('moiap13')
  const [password, setPassword] = useState('password')
  const security = passwordSecurity(password)

  return <div className="container my-3 vstack gap-2">
    <Input
      label='Username'
      placeholder='Username'
      value={username}
      onChange={setUsername}
    />
    <div>  
      <Input
        label='Password'
        placeholder='Password'
        value={password}
        onChange={setPassword}
      />
      <div>Password security: {security}</div>
    </div>
  </div>

function passwordSecurity(password) {
  if (password.length < 8) return 'Weak'
  if (password.length < 12) return 'Moderate'
  return 'Strong'
}
export default App
```

Le problème avec ce code est que la fonction `passwordSecurity` est appelée à chaque rendu, même si la valeur de `password` n'a pas changé. Cela peut être un problème si la fonction est coûteuse en ressources.

# Initial code - weak performance simulation

```jsx
import { useState, useEffect, useMemo } from 'react'
import { Input } from './components/forms/Input'
import { Checkbox } from './components/forms/Checkbox'

function App() {
  const [username, setUsername] = useState('moiap13')
  const [password, setPassword] = useState('password')
  const security = passwordSecurity(password)

  return <div className="container my-3 vstack gap-2">
    <Input
      label='Username'
      placeholder='Username'
      value={username}
      onChange={setUsername}
    />
    <div>  
      <Input
        label='Password'
        placeholder='Password'
        value={password}
        onChange={setPassword}
      />
      <div>Password security: {security}</div>
    </div>
  </div>

function passwordSecurity(password) {
  // simulate a slow function
  let start = performance.now()
  while (performance.now() - start < 100) {}

  if (password.length < 8) return 'Weak'
  if (password.length < 12) return 'Moderate'
  return 'Strong'
}
export default App
```

Ici on voit que la fonction `passwordSecurity` est couûteuse en ressources et rallentit toute notre application.

Quand on rentre le `password`, on est obligé d'attendre que la fonction se termine pour voir le résultat. Mais ce même comportement est observer quand on change le `username` alors que la fonction `passwordSecurity` retourne la même valeur qu'avant car le `password` n'a pas changé.

# Solution

```jsx
import { useState, useEffect, useMemo } from 'react'
import { Input } from './components/forms/Input'
import { Checkbox } from './components/forms/Checkbox'

function App() {
  const [username, setUsername] = useState('moiap13')
  const [password, setPassword] = useState('password')
  // Ici on utilise le hook useMemo pour mémoriser le résultat de la fonction passwordSecurity
  // Contrairement à useEffect, useMemo retourne le résultat de la fonction, que je stoque dans la variable security
  const security = useMemo(() => passwordSecurity(password), [password])

  return <div className="container my-3 vstack gap-2">
    <Input
      label='Username'
      placeholder='Username'
      value={username}
      onChange={setUsername}
    />
    <div>  
      <Input
        label='Password'
        placeholder='Password'
        value={password}
        onChange={setPassword}
      />
      <div>Password security: {security}</div>
    </div>
  </div>

function passwordSecurity(password) {
  // simulate a slow function
  let start = performance.now()
  while (performance.now() - start < 100) {}

  if (password.length < 8) return 'Weak'
  if (password.length < 12) return 'Moderate'
  return 'Strong'
}
export default App
```

Ici l'application sera toujours lente quand on changera le password, mais elle ne le sera plus quand on changera le username. Car tant que la `dépendance` de `useMemo` ne change pas, le résultat de la fonction ne sera pas recalculé.

# Pourquoi ne pas toujours utiliser `useMemo` ?

Avant d'utiliser `useMemo`, il faut se demander si une **computation** est executée plus osuvent que nécessaire. Si c'est le cas, alors `useMemo` est une bonne solution. Mais si la computation est executée à chaque rendu, alors `useMemo` n'est pas la solution.

De manière générale, on va `"memoizer"` les fonction qui sont coûteuse, et qui ne changent pas souvent. Si une fonction s'execute vite et change souvent, alors `useMemo` n'est pas la solution.

# React comments

En général, quand un calcul prend trop de temps à s'executer, `React` nous le dis:
- Dans la console de `chrome` on peut choisir d'afficher en `verbose`
- On peut voir des messages de Log de `Vite`
- Mais aussi des messages de `React` qui nous disent que le rendu est trop long
- Ces messages de `React` peuvent être un bon indicateur pour savoir si une optimisation est nécessaire

# useMemo - onMounted

Comme pour `useEffect`, `useMemo` peut être utilisé pour executer une fonction au premier rendu. Pour cela, on passe un tableau vide comme deuxième argument.

```jsx
const today = useMemo(() => new Date(), [])
```

Ici, `today` sera la date du premier rendu, et ne changera pas par la suite.

On peut utiliser cette approche quand on génère un nombre aléatoire, ou quand on veut mémoriser une valeur qui ne changera pas tant que le composant est vivant.

# Le `hook` useId

Le `hook` `useId` permet de générer un `id` unique pour un élément `html`. Il est utile pour les formulaires, ou pour les éléments `html` qui ont besoin d'un `id`.

# useId

```jsx
export function Input({placeholder, value, onChange, label}) {
  const randomId = Math.random().toString(36).substring(7);
  return <div>
    {label && <label className="form-label" htmlFor={randomId}>{label}</label>}
    <input
      id={randomId}
      type="text" 
      className="form-control"
      placeholder={placeholder} 
      value={value} 
      onChange={e => onChange(e.target.value);}
    />
  </div>
}
```

# useId

```jsx
export function Input({placeholder, value, onChange, label}) {
  const id = useId();
  return <div>
    {label && <label className="form-label" htmlFor={id}>{label}</label>}
    <input
      id={id}
      type="text" 
      className="form-control"
      placeholder={placeholder} 
      value={value} 
      onChange={e => onChange(e.target.value);}
    />
  </div>
}
```

# Conclusion

- `useMemo` permet de mémoriser le résultat d'une fonction pour éviter de la recalculer à chaque rendu
- `useMemo` est utile pour optimiser les performances de votre application React
  - **/!\ n'essayez pas de faire de l'optimization préventive**
- `useMemo` est utile pour mémoriser des valeurs qui ne changent pas souvent
- `useMemo` peut être utilisé pour executer une fonction au premier rendu
- `useMemo` est coûteux en ressources, et ne doit être utilisé que si nécessaire
- `useId` permet de générer un `id` unique pour un élément `html`
- `useId` est utile pour les formulaires, ou pour les éléments `html` qui ont besoin d'un `id`
- `useId` est utile pour éviter de générer des `id` aléatoires à chaque rendu

Ici le problème est que le scroll event listener n'est jamais retiré. Quand le composant "disparait", le listener continue à écouter les événements.

Pour remedier a ça, on peut retourner une fonction dans le `useEffect` qui sera appelée lors du démontage du composant.

# useEffect - clean events listeners

On peut utiliser le `hook` `useEffect` pour effectuer des actions lors du montage du composant.

Quand on ne specifie pas de dépendances le `hook` `useEffect` est appelé une seule fois lors du montage du composant. on appelle ce moment `onMounted`.

```jsx
function EditTitle() {
  const [title, setTitle] = useState('')
  const [firstname, setFirstname] = useState('')
  const [y, setY] = useState(0)

  useEffect(() => {
    const scrollHandler = () => {
      console.log('scroll')
      setY(window.scrollY)
    }
    window.addEventListener('scroll', scrollHandler)
    
    return () => {
      window.removeEventListener('scroll', scrollHandler)
    }
  }, [])

  useEffect(() => {
    document.title = title
  }, [title])

  return <div className='vstack gap-3'>
    <div>Y: {y}</div>
    <Input
      placeholder='Modify Title'
      value={title}
      onChange={setTitle}
    />
    <Input
      placeholder='Firstname'
      value={firstname}
      onChange={setFirstname}
    />
  </div>
}
```

**Note: on peut aussi utiliser la callback de nettoyage quand il y a une dépendance, dans ce cas, quand la dépendance change, la fonction de nettoyage est d'abord appellée et ensuite la callback `useEffect`**

# useEffect - Pro Tips

Le `hook` `useEffect` est utilisé pour les **effets de bords** comme brancher des éléments globaux ou modifier des trucs qui "n'appartiennent pas" au composant (qui sont en dehors).

```jsx
useEffect(() => {
  setY(window.scrollY)
}, [title])
```

l'example ci dessus est un mauvais exemple, car on ne devrait pas utiliser le `useEffect` pour mettre à jour une variable d'état. On devrait utiliser le `useState` pour ça. *De manière générale, quand on a un `setter` en première instruction d'un `useEffect`, c'est qu'on fait quelque chose de mal, il y a une autre façon de le faire*

# useEffect - Pro Tips

```jsx
function App() {
  const [duration, setDuration] = useState(5)
  const [secondsLeft, setSecondsLeft] = useState(duration)

  console.log('render')

  useEffect(() => {
    setSecondsLeft(secondsLeft - 1)
  }, [duration])

  return <div className='vstack gap-2'>
    <Input
      placeholder='Duration'
      value={duration}
      onChange={setDuration}
    />
    <div>Seconds Left: {secondsLeft}</div>
  </div>
}
```

Ici le problème est que le `useEffect` est appellé à chaque fois que `duration` change, or le changement de `duration` re-rend le composant, ce qui re-appelle le `useEffect` qui le re-rend a son tour. Le composant sera re-rendu deux fois au lieu d'une seule.

# useEffect - Pro Tips

```jsx
function App() {
  const [duration, setDuration] = useState(5)
  const [secondsLeft, setSecondsLeft] = useState(duration)

  console.log('render')

  const handleChange = (v) => {
    setDuration(v)
    setSecondsLeft(v)
  }

  useEffect(() => {
    const interval = setInterval(() => {
      setSecondsLeft(v => v - 1)
    }, 1000)
    return () => {
      clearInterval(interval)
    }
  }, [duration])

  return <div className='vstack gap-2'>
    <Input
      placeholder='Duration'
      value={duration}
      onChange={handleChange}
    />
    <div>Seconds Left: {secondsLeft}</div>
  </div>
}
```

# TP - Correction 

```jsx
function App() {
  const [duration, setDuration] = useState(5)
  const [secondsLeft, setSecondsLeft] = useState(duration)

  console.log('render')

  const handleChange = (v) => {
    setDuration(v)
    setSecondsLeft(v)
  }

  useEffect(() => {
    const interval = setInterval(() => {
      setSecondsLeft(v => {
        if (v === 0) {
          clearInterval(interval)
          return 0
        }
        return v - 1
      })
    }, 1000)
    return () => {
      clearInterval(interval)
    }
  }, [duration])

  return <div className='vstack gap-2'>
    <Input
      placeholder='Duration'
      value={duration}
      onChange={handleChange}
    />
    <div>Seconds Left: {secondsLeft}</div>
  </div>
}
```

# useEffect - Conclusion

Le `hook` `useEffect` est utilisé pour gérer les effets de bord dans les composants. Les effets de bord sont des actions qui sont effectuées en dehors du cycle de vie normal des composants React. comme par exemple:
- Brancher des éléments globaux
- setTimeout, setInterval
- Fetch des données

Penser a utiliser la callback de retour pour nettoyer vore `useEffect`.

**De manière générale, on essaiera d'utiliser le moins possible `useEffect`, voyez `useEffect` comme une solution de dernier recours**

[you might not need an effect](https://react.dev/learn/you-might-not-need-an-effect)